In [2]:
!pip install torch torchvision torchaudio

In [3]:
from google.colab import files
uploaded = files.upload()


Saving Final_cleaned_code_dataset.csv to Final_cleaned_code_dataset.csv


In [4]:
!pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.7 MB/s eta 0:00:00


#### 1. import libraries

In [5]:
import pandas as pd
import torch
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import evaluate

#### 2. Load & Prepare Dataset

In [6]:
df = pd.read_csv("Final_cleaned_code_dataset.csv")

In [7]:
import ast
df['tags'] = df['tags'].apply(ast.literal_eval)

# Combine code + tags as input for CodeT5
df['input_text'] = df['code'] + " " + df['tags'].apply(lambda x: " ".join(x))

# Encode TC labels
le = LabelEncoder()
df['tc_label'] = le.fit_transform(df['complexity'])

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.2, seed=42)

#### 3. Load CodeT5 Model & Tokenizer

In [8]:
model_name = "Salesforce/codet5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(le.classes_)
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at Salesforce/codet5-small and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### 4. Tokenize Dataset

In [9]:
def tokenize(batch):
    tokens = tokenizer(batch['input_text'], padding='max_length', truncation=True, max_length=512)
    tokens['labels'] = batch['tc_label']  # this is critical
    return tokens

tokenized_datasets = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/2647 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

Map:   0%|          | 0/662 [00:00<?, ? examples/s]

In [10]:
print(tokenized_datasets['train'].column_names)

['code', 'complexity', 'tags', 'code_length', 'input_text', 'tc_label', 'input_ids', 'attention_mask', 'labels']


#### 5. Define Metrics

In [11]:
import evaluate
import numpy as np

accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

In [12]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred.predictions, eval_pred.label_ids

    # If logits is a tuple (sometimes happens)
    if isinstance(logits, tuple):
        logits = logits[0]

    predictions = np.argmax(logits, axis=-1)

    acc = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")

    return {"accuracy": acc["accuracy"], "f1": f1["f1"]}

#### 6. Training Arguments

In [13]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./codet5_tc",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=5e-5,
    weight_decay=0.01,
    save_total_limit=2,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)


#### 7. Fine Tuning(Training)

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

/tmp/ipython-input-2511900202.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: somrima09072003 (somrima09072003-indian-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.644637,0.737160,0.737399
2,0.971900,0.539106,0.774924,0.778957
3,0.971900,0.443171,0.818731,0.820142


There were missing keys in the checkpoint model loaded: ['transformer.encoder.embed_tokens.weight', 'transformer.decoder.embed_tokens.weight'].


TrainOutput(global_step=993, training_loss=0.7584779553062846, metrics={'train_runtime': 669.7178, 'train_samples_per_second': 11.857, 'train_steps_per_second': 1.483, 'total_flos': 1081219246009344.0, 'train_loss': 0.7584779553062846, 'epoch': 3.0})

In [15]:
model.save_pretrained("./model_codeT5")
tokenizer.save_pretrained("./model_codeT5")

('./model_codeT5/tokenizer_config.json',
 './model_codeT5/special_tokens_map.json',
 './model_codeT5/vocab.json',
 './model_codeT5/merges.txt',
 './model_codeT5/added_tokens.json',
 './model_codeT5/tokenizer.json')

In [17]:
import shutil
shutil.make_archive("model_codeT5", 'zip', "./model_codeT5")

'/content/model_codeT5.zip'

In [18]:
from google.colab import files
files.download("model_codeT5.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>